In [ ]:
from selenium import webdriver #webdriver is what launches your browers
from selenium.webdriver.common.by import By #imports By, By is the module selenium uses to search for controls
from selenium.webdriver.support.ui import WebDriverWait, Select #webdriverwait is to pace the script, tell its to wait for page to load.
from selenium.webdriver.support import expected_conditions as EC #EC is an expected outcome, if outcome is met then code proccedes
import pandas as pd #Imports Pandas for dataframes


driver = webdriver.Chrome() #this line opens your browers, in this case chrome
driver.get('https://valuation2012.durban.gov.za/') #opens the page in chrome


driver.switch_to.frame("mainFrame") #On this page controls are kept in 'Frames' to access certain controls you have to switch to the frame the control is in


valuation_roll_dropdown = WebDriverWait(driver, 10).until(  #the dropdown box on the 1st page that selects Full Title or Sectional Title, wait 10 seconds until it loads
    EC.presence_of_element_located((By.ID, "drpSearchType"))  #look for expected come by The dropdown box ID. Selenium can use Properties like ID, NAME, XPATH etc
)
select = Select(valuation_roll_dropdown)
select.select_by_visible_text("Full Title Property")  # Select the valuation type


go_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "btnGo"))
)
driver.execute_script("arguments[0].click();", go_button)  # Click the Go button this loads the search parameters


driver.switch_to.frame("frmSearchCriter") #Switch to the frame where the search parameters are

dropdown_element = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, "//table//select[@id='drpVolumeNo']")) #this is the dropdown box where i selected 01 to get the output table, this can be set to any other value to get a different output table
)

dropdown = Select(dropdown_element)
dropdown.select_by_visible_text("01") #Select 01

search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[3]/table/tbody/tr[1]/td[2]/table/tbody/tr/td[1]/input")) #uses XPATH to find control
)
driver.execute_script("arguments[0].click();", search_button)  # Click the Search button


driver.switch_to.default_content() #this line takes us back to the default frame


WebDriverWait(driver, 15).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "mainFrame"))) #to get to the table which is where our data is we have to go through a nested frame
print("Switched to mainFrame") #We start at mainFrame

  



WebDriverWait(driver, 30).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "frameSearch")))
print("Switched to frameSearch") #and then to frameSearch this is where our table and data is

  
table = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'Grid') or contains(@border, '1')]")) #uses Xpath to find table
    )
print("Table located")


rows = table.find_elements(By.TAG_NAME, "tr")
data = []
for row in rows[2:]:
    cols = row.find_elements(By.TAG_NAME, "td") or row.find_elements(By.TAG_NAME, "th")
    data.append([col.text.strip() for col in cols]) #now that we found the table we can store the rows and columns in our data list

data = [row for row in data if any(cell.strip() for cell in row)]

num_columns = len(data[0])
valid_data = [row for row in data if len(row) == num_columns]
first_200_data = valid_data[:201]    
df = pd.DataFrame(first_200_data[1:], columns=first_200_data[0]) #Because theres so much data the script may take a few moments to complete, to improve this load issue i saved the 1st 200 rows only in the dataframe +-40 seconds

print(df.head(200))
df.to_csv(r'C:\Users\Kams\Desktop\tech-challenge-main\Full Title Property Data.csv', index=False) #saves the dataframe as a csv 
print("First 200 records saved to 'valuation_data_first_200.csv'")

Switched to mainFrame
Switched to frameSearch
Table located
    Rate Number                 Legal Description                     Address  \
0      00008808               ERF 10031 of DURBAN          2 POINT ROAD,POINT   
1      00008909               ERF 10073 of DURBAN       92 BELL STREET, POINT   
2      00009010               ERF 10054 of DURBAN  179 ESCOMBE TERRACE ,POINT   
3      00009111               ERF 10054 of DURBAN  183 ESCOMBE TERRACE ,POINT   
4      00009515  Portion 3 of ERF 10031 of DURBAN         76 POINT ROAD,POINT   
..          ...                               ...                         ...   
195    00628038  Portion 3 of ERF 11013 of DURBAN        424 SMITH STREET,CBD   
196    00635007        REM of ERF 11014 of DURBAN         437 WEST STREET,CBD   
197    00635007               ERF 11016 of DURBAN        414 SMITH STREET,CBD   
198    00635613        REM of ERF 11018 of DURBAN      GREENACRES PASSAGE,CBD   
199    00636017  Portion 4 of ERF 11017 of DURBAN